# Transfer Learning using TAO Toolkit and Roboflow

___

**Downloading the dataset from roboflow**

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="API_KEY")
project = rf.workspace("WORKSPACE_NAME").project("PROJECT_NAME")
dataset = project.version(4).download("coco")

## Setup TAO mounts.

Since the TAO Toolkit abstracts operations into Docker containers, we need to specify how the Docker containers can access outside data. We do this with a file called `~/.tao_mounts.json` where we specify the mapping from "source" directories to "destination" directories. See more here: https://docs.nvidia.com/tao/tao-toolkit/text/tao_launcher.html#running-the-launcher

In [3]:
import os

%env KEY=tlt_encode
%env NUM_GPUS=1
%env USER_EXPERIMENT_DIR=/workspace/project/detectnet_v2
%env DATA_DOWNLOAD_DIR=/workspace/project/data

os.environ["LOCAL_PROJECT_DIR"] = "PATH_TO_YOUR_PROJECT_FILE"

os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "Auto-Robert-2"
)

env: KEY=tlt_encode
env: NUM_GPUS=1
env: USER_EXPERIMENT_DIR=/workspace/project/detectnet_v2
env: DATA_DOWNLOAD_DIR=/workspace/project/data


In [4]:
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")
drive_map = {
    "Mounts": [
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/project"
        },
    ],
    "DockerOptions": {
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [7]:
!tao bpnet run ls /workspace/project/Auto-Robert-2/test

2022-12-05 21:12:56,892 [INFO] root: Registry: ['nvcr.io']
2022-12-05 21:12:56,964 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.22.05-tf1.15.5-py3
2022-11-10-182659_jpg.rf.cafe776877ae513adfbd414cd3ecdb8d.jpg
2022-11-10-182704_jpg.rf.0ddc10339b6b9298afa228b5f1cc635a.jpg
2022-11-10-182734_jpg.rf.2bd3a781d4c182e62fceeb64f2dffbb7.jpg
2022-11-10-182749_jpg.rf.26c9403b5b737b5f18ee5d1c83bb4519.jpg
2022-11-10-182826_jpg.rf.21d4d06a79295789e2b3286d558d4803.jpg
2022-11-10-182844_jpg.rf.d73f54cfd0b551c102efea410ab1ddfb.jpg
2022-11-10-183013_jpg.rf.0c534e1ff3391b6fc047a8aa811a875f.jpg
2022-11-10-183015_jpg.rf.d451d8d5099392c8c73241485976446d.jpg
2022-11-10-183036_jpg.rf.cfff946a867a35815905a3aea1db7c74.jpg
2022-11-10-183055_jpg.rf.a98f9c8f9710a684525e08c08d48166a.jpg
2022-11-10-183122_jpg.rf.47059d18da432dcea0d9979cff52d3c9.jpg
2022-11-10-183123_jpg.rf.ece56c377b08eaffc5030d6b5d17ff34.jpg
2022-11-10-183152_jpg.rf.978fe06

In [8]:
!tao detectnet_v2 dataset_convert -d "/workspace/project/coco_config.json" --gpu_index 0 -o "/workspace/project/output.tfrecords"

2022-12-05 21:13:29,322 [INFO] root: Registry: ['nvcr.io']
2022-12-05 21:13:29,368 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.22.05-tf1.15.4-py3
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-bm86c2ua because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Using TensorFlow backend.
2022-12-06 00:13:36,260 [INFO] iva.detectnet_v2.dataio.build_converter: Instantiating a coco converter
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
load

In [9]:
!mv output.tfrecords* output_tfrecords

In [ ]:
!ngc registry model download-version nvidia/tao/pretrained_detectnet_v2:resnet18 --dest $LOCAL_EXPERIMENT_DIR/pretrained_resnet18

In [10]:
!tao detectnet_v2 train -e "/workspace/project/experiment_spec.txt" -r "/workspace/project/resnet18_unpruned" -k "roboflow" --use_amp

2022-12-05 21:18:44,484 [INFO] root: Registry: ['nvcr.io']
2022-12-05 21:18:44,527 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.22.05-tf1.15.4-py3
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-mrfl9a3q because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Using TensorFlow backend.

2022-12-06 00:18:49,580 [WARNING] tensorflow: From /root/.cache/bazel/_bazel_root/b770f990bb7b9e2db5771981fb3a38b4/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_whe

In [11]:
!tao detectnet_v2 evaluate -e /workspace/project/experiment_spec.txt\
                           -m /workspace/project/resnet18_unpruned/weights/model.tlt \
                           -k "roboflow"

2022-12-05 22:50:05,003 [INFO] root: Registry: ['nvcr.io']
2022-12-05 22:50:05,051 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.22.05-tf1.15.4-py3
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-pv105bxu because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Using TensorFlow backend.

2022-12-06 01:50:11,755 [WARNING] tensorflow: From /root/.cache/bazel/_bazel_root/b770f990bb7b9e2db5771981fb3a38b4/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_whe

In [12]:
!tao detectnet_v2 prune \
                  -m /workspace/project/resnet18_unpruned/weights/model.tlt \
                  -o /workspace/project/resnet18_pruned/model_pruned.tlt \
                  -eq union \
                  -pth 0.0000052 \
                  -k "roboflow"

2022-12-05 22:51:26,598 [INFO] root: Registry: ['nvcr.io']
2022-12-05 22:51:26,643 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.22.05-tf1.15.4-py3
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-64st9ww6 because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Using TensorFlow backend.
2022-12-06 01:51:33,610 [INFO] modulus.pruning.pruning: Exploring graph for retainable indices
2022-12-06 01:51:33,953 [INFO] modulus.pruning.pruning: Pruning model and appending pruned node

In [13]:
!tao detectnet_v2 train -e /workspace/project/detectnet_v2_retrain_resnet18.txt \
                        -r /workspace/project/experiment_dir_retrain \
                        -k "roboflow" \
                        -n resnet18_detector_pruned 

2022-12-05 22:51:46,653 [INFO] root: Registry: ['nvcr.io']
2022-12-05 22:51:46,699 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.22.05-tf1.15.4-py3
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-w63ccxfo because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Using TensorFlow backend.

2022-12-06 01:51:51,723 [WARNING] tensorflow: From /root/.cache/bazel/_bazel_root/b770f990bb7b9e2db5771981fb3a38b4/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_whe

In [14]:
!tao detectnet_v2 evaluate -e /workspace/project/detectnet_v2_retrain_resnet18.txt \
                           -m /workspace/project/experiment_dir_retrain/weights/resnet18_detector_pruned.tlt \
                           -k "roboflow"

2022-12-06 01:07:42,054 [INFO] root: Registry: ['nvcr.io']
2022-12-06 01:07:42,097 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.22.05-tf1.15.4-py3
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-8rosor7j because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Using TensorFlow backend.

2022-12-06 04:07:48,983 [WARNING] tensorflow: From /root/.cache/bazel/_bazel_root/b770f990bb7b9e2db5771981fb3a38b4/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_whe

In [15]:
!tao detectnet_v2 inference -e /workspace/project/detectnet_v2_inference.txt \
                            -o /workspace/project/tlt_infer_testing \
                            -i /workspace/project/Auto-Robert-2/test \
                            -k "roboflow"

2022-12-06 01:09:03,900 [INFO] root: Registry: ['nvcr.io']
2022-12-06 01:09:03,938 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.22.05-tf1.15.4-py3
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-zbx55l7d because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
INFO: Merging specification from /workspace/project/detectnet_v2_inference.txt
INFO: Creating output inference directory
INFO: Overlain images will be saved in the output path.
INFO: Constru

In [18]:
!tao detectnet_v2 calibration_tensorfile -e /workspace/project/detectnet_v2_retrain_resnet18.txt \
                                         -m 10 \
                                         -o /workspace/project/experiment_dir_final/calibration.tensor

2022-12-06 01:11:11,164 [INFO] root: Registry: ['nvcr.io']
2022-12-06 01:11:11,219 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.22.05-tf1.15.4-py3
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-bt9maskv because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Using TensorFlow backend.
2022-12-06 04:11:16,710 [INFO] __main__: This method is soon to be deprecated. Please use the -e option in the export command to instantiate the dataloader and generate samples for calibrati

In [19]:
!rm -rf /workspace/project/experiment_dir_final/resnet18_detector.etlt
!rm -rf /workspace/project/experiment_dir_final/calibration.bin
!tao detectnet_v2 export \
                  -m /workspace/project/experiment_dir_retrain/weights/resnet18_detector_pruned.tlt \
                  -o /workspace/project/experiment_dir_final/resnet18_detector.etlt \
                  -k "roboflow"  \
                  --cal_data_file /workspace/project/experiment_dir_final/calibration.tensor \
                  --data_type int8 \
                  --batches 10 \
                  --batch_size 4 \
                  --max_batch_size 4\
                  --engine_file /workspace/project/experiment_dir_final/resnet18_detector.trt.int8 \
                  --cal_cache_file /workspace/project/experiment_dir_final/calibration.bin \
                  --verbose

2022-12-06 01:11:24,780 [INFO] root: Registry: ['nvcr.io']
2022-12-06 01:11:24,830 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.22.05-tf1.15.4-py3
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-dfz1ha06 because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2022-12-06 04:11:31,948 [INFO] iva.common.export.keras_exporter: Using input nodes: ['input_1']
2022-12-06 04:11:31,948 [INFO] iva.common.export.keras_exporter: Using output nodes: ['output_

In [20]:
!tao converter /workspace/project/experiment_dir_final/resnet18_detector.etlt \
                   -k "roboflow" \
                    -c /workspace/project/experiment_dir_final/calibration.bin \
                   -o output_cov/Sigmoid,output_bbox/BiasAdd \
                   -d 3,640,640 \
                   -i nchw \
                   -t int8 \
                   -b 4 \
                   -m 4 \
                   -e /workspace/project/experiment_dir_final/resnet18_int8_detector.engine

2022-12-06 01:12:36,191 [INFO] root: Registry: ['nvcr.io']
2022-12-06 01:12:36,238 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.22.05-tf1.15.5-py3
[INFO] [MemUsageChange] Init CUDA: CPU +473, GPU +0, now: CPU 484, GPU 330 (MiB)
[INFO] [MemUsageSnapshot] Begin constructing builder kernel library: CPU 484 MiB, GPU 330 MiB
[INFO] [MemUsageSnapshot] End constructing builder kernel library: CPU 638 MiB, GPU 372 MiB
[INFO] Reading Calibration Cache for calibrator: EntropyCalibration2
[INFO] Generated calibration scales using calibration cache. Make sure that calibration cache has latest scales.
[INFO] To regenerate calibration cache, please delete the existing one. TensorRT will generate a new calibration cache.
[WARNING] Missing scale and zero-point for tensor output_bbox/bias, expect fall back to non-int8 implementation for any layer consuming or producing given tensor
[WARNING] Missing scale and zero-point for te